For Basque:

In [3]:
# coding=utf-8
import nltk
import sys
import re
import os

##################WORDFRECUENCY###################################################
# load a freq file into a dictionary
import codecs
d = {}
#f = codecs.open('compounds.dat', encoding='utf-8')
with codecs.open('LB2014Maiztasunak_zenbakiakKenduta.csv',encoding='utf-8') as f:
    next(f)
    for line in f:
       (val,key) = line.split(",")
       d[key] = val
#for key,val in d.items():
 #   print (key, "=>", val)
    
def zipf_frequency_eu(lemma):
    if d.get(lemma):
        return (int(d.get(lemma)))
    else:
        return 1
#################WORDNET##########################
#Instalar 'wordnet' 
nltk.download('wordnet')
#Add multilingual wordnet
nltk.download('omw')
from nltk.corpus import wordnet as wn
def set_is_empty(some_set):
    return some_set == set()

###################WSD####################################################
#simple_lesk erabiltzeko
#nltk.download('averaged_perceptron_tagger')
#from pywsd.lesk import simple_lesk
from nltk.wsd import lesk
###############ImageNet###########################
#from bs4 import BeautifulSoup
#import requests
#def imagenet_offset2url(offset):
#    soffset=str(offset).zfill(8)
#    query="http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n"+str(soffset)
#    page = requests.get(query)#ship synset
    # BeautifulSoup is an HTML parsing library
    #puts the content of the website into the soup variable, each url on a different line
#    soup = BeautifulSoup(page.content, 'html.parser')
#    str_soup=str(soup)#convert soup to string so it can be split
#    split_urls=str_soup.split('\r\n')#split so each url is a different possition on a list
#    return(split_urls[0])
###############wikipedia############################
import wikipedia
def wikipedia_offset2url(text):
#WIKIPEDIA
    url= ''
    ext = ''
    svg = False
    i = 0
    while (svg==False):
        try:
            #First of all wikipedia.page() will store all the relevant informations 
            #from the requested page in the variable imagepage. 
            imagepage=wikipedia.page(text)
            #Existen muchas funciones. imagepage.url devuelve la url, imagepage.url.content el contenido, title el título....
            #imagepage.images[0] will return the URL of the image that is present at index 0. 
            #If you want to fetch another image use index as 1, 2, 3, etc, according to images present in the page.
            url = imagepage.images[i]
            #print(url)
            name, ext =(url.split('/')[-1].split('.'))
            if ext=='.svg':
                  i = i+1
            else:
                  svg = True
        except:
            url= ''
            svg = True
    return url
                #si no desambiguamos
##########Wikidata###################################
#Wikidata query service -> code in python
#pip install wikidata
#https://pypi.org/project/Wikidata/
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/
import wikidata
from SPARQLWrapper import SPARQLWrapper, JSON
from wikidata.client import Client
def get_results(endpoint_url, query):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()
def wikidata_offset2url(offset):
    #WIKIDATA
    endpoint_url = "https://query.wikidata.org/sparql"
    soffset=str(offset).zfill(8)
    query="Select * where {?item wdt:P2888 <http://wordnet-rdf.princeton.edu/wn30/"+soffset+"-n> }"
    try:
        results = get_results(endpoint_url, query)
        ema=results["results"]["bindings"]
        item=ema[0].get('item')
        value=item.get('value')
        head, tail = value.split("http://www.wikidata.org/entity/")
        #for result in results["results"]["bindings"]:
          #    print(result)
        client = Client()
        entity = client.get(tail, load=True)
        image_prop = client.get('P18')
        image = entity[image_prop]
        #image-><wikidata.commonsmedia.File 'File:KBS "The Producers" press conference, 11 May 2015 10.jpg'>
        #image.image_resolution
        url=image.image_url
    except:
        url=''
    return url
#####################################################
##############STOPWORDS################################################################################
#Generating a word cloud with no optional parameters based on the above string:
#from wordcloud import STOPWORDS
#stopwords = set(STOPWORDS)
#from nltk.corpus import stopwords
#stopwords = set(stopwords.words("english"))
#words = [w for w in words if not w in stopwords]
#stopwords=set()
#with open("stopwords_formak.txt") as file:
#    for line in file:
#        line = line.strip() #preprocess line
#        stopwords.add(line)
#print (stopwords)
stopwords= set(line.strip() for line in open('stopwords_formaketakonektoreak.txt'))
stopwords.add("ondo")
stopwords.add("ordu")
stopwords.add("jarraian")
stopwords.add("igaro")
#Gehitu konektoreak
#stopwords={'to', 'of', 'us'}


###################PARSER#################################################
import stanfordnlp #->/media/datos/Dropbox/ikerkuntza/metrix-env/lib/python3.6/site-packages/stanfordnlp
MODELS_DIR = '/home/kepa/eu'
#stanfordnlp.download('eu', MODELS_DIR) # Download the English models
config = {'processors': 'tokenize,pos,lemma,depparse', # Comma-separated list of processors to use
'lang': 'eu', # Language code for the language to build the Pipeline in
'tokenize_model_path': '/home/kepa/eu/eu_bdt_models/eu_bdt_tokenizer.pt', # Processor-specific arguments are set with keys "{processor_name}_{argument_name}"
'pos_model_path': '/home/kepa/eu/eu_bdt_models/eu_bdt_tagger.pt',
'pos_pretrain_path': '/home/kepa/eu/eu_bdt_models/eu_bdt.pretrain.pt',
'lemma_model_path': '/home/kepa/eu/eu_bdt_models/eu_bdt_lemmatizer.pt',
'depparse_model_path': '/home/kepa/eu/eu_bdt_models/eu_bdt_parser.pt',
'depparse_pretrain_path': '/home/kepa/eu/eu_bdt_models/eu_bdt.pretrain.pt'
}
parser = stanfordnlp.Pipeline(**config)
####input file##################
input="errezeta.txt" #sys.argv[1]

####nivel de dificultad#########
difficult = 100000 #int(sys.argv[2])

####ouput files#################
#estadisticos
estadisticaoutput=input+".out.csv"

#Write all the information in the file
estfile = open(estadisticaoutput, "w")


###############Tratamiento de texto###############################################
#quitar todos los retornos \n si contiene
text = open(input).read().replace('\n', '')
#remove text inside parentheses
#text = re.sub(r'\([^)]*\)', '', text)
#separa , . ! ( ) ? ; del texto con espacios, teniendo en cuenta que los no son numeros en el caso de , y . 
text = re.sub(r"\_", " ", text)
text = re.sub(r"\-", " ", text)
text = re.sub(r'[.]+(?![0-9])', r' . ', text)
text = re.sub(r'[,]+(?![0-9])', r' , ', text)
text = re.sub(r"!", " ! ", text)
text = re.sub(r"\(", " ( ", text)
text = re.sub(r"\)", " ) ", text)
text = re.sub(r"\?", " ? ", text)
text = re.sub(r";", " ; ", text)
#sustituye 2 espacios seguidos por 1
text = re.sub(r"\s{2,}", " ", text)

################WORDCLOUD################################################
from wordcloud import WordCloud
#Generating a word cloud with no optional parameters based on the above string:
# #This is because the wordcloud module ignores stopwords by default. Refer to Part 1 of the NLTK tutorial if the concept of stopwords is new to you.If we wish, we can specify our own set of stopwords, instead of the stopwords provided by default.
# #Con relative_scaling = 0, solo se consideran los rangos de las palabras. Si modificamos esto a relative_scaling = 1.0, entonces una palabra que aparece dos veces más frecuentemente aparecerá dos veces el tamaño. Por defecto, relative_scaling = 0.5.
# wordcloud = WordCloud(relative_scaling=1.0, stopwords={'to', 'of'}).generate(text)
wordcloud = WordCloud(relative_scaling=1.0,stopwords=stopwords).generate(text)
#Finally, use matplotlib to render the word cloud:
#plot_wordcloud(wordcloud)
wordcloudfilename=input+".png"
#wordcloudfilename="resume.png"
wordcloud.to_file(wordcloudfilename)
###################analizador morfosintactico#################################################
palabras_diferentes = []
doc=parser(text)
for sent in doc.sentences:
    #Por cada sentencia
    for entry in sent.words:
        definition = ''
        synonyms = []
        offset = 0
        nueve = '_' 
        url = ''              
        #Por cada palabra
        if entry.text.isalpha() and (entry.upos == 'NOUN' or entry.upos == 'VERB') and entry.text.lower() not in palabras_diferentes:
            #Si la palabra es un content word
            palabras_diferentes.append(entry.text.lower())
            lemafrequency = zipf_frequency_eu(entry.lemma)
            #<=6 zaila
            #<=34 erdiko zailtasuna
            #<=100000 errazak
            if lemafrequency <= int(difficult):
                lema=entry.lemma
                synset_ids = wn.synsets(lema,lang='eus')
                if entry.upos=='NOUN':
                    patron='n'
                if entry.upos=='VERB':
                    patron='v'
                #intentamos obtener la synset desambiguada, y la def,exam,sin and img-url sobre esa synset
                try:
                    synset_desambiguado = lesk(text.split(), entry.text, pos=patron, synsets=synset_ids)
                    #definicion
                    definition = synset_desambiguado.definition()
                    #ejemplos
                    examples = synset_desambiguado.examples()
                    #offset
                    offset = synset_desambiguado.offset()
                    #traduccion es
                    traduccion_es=answer2.lemmas('spa')
                    es_list=[]
                    for l in traduccion_es:
                        es_list.append(l.name())
                    #traduccion eu
                    traduccion_eu=answer2.lemmas('eus') 
                    eu_list=[]
                    for l in traduccion_eu:
                        eu_list.append(l.name())
                    #sinonimos
                    for l in synset_desambiguado.lemma_names():
                        synonyms.append(l.lower())
                    try:
                        synonyms.remove(entry.text.lower())
                    except:
                        pass
                    try:
                        synonyms.remove(entry.lemma)
                    except:
                        pass

                    #url imagen
                    if entry.upos=='NOUN':
                        #deia wikidata
                        url=wikidata_offset2url(offset)
                        if url == '':
                            url=wikipedia_offset2url(entry.text)
                #si no desambiguamos#PRIMER SYNSET FILTRADO POR CATEGORIA
                except:
                    count = 0
                    for synset in synset_ids:
                        if entry.upos=='NOUN':
                            patron='.n.'
                        if entry.upos=='VERB':
                            patron='.v.'

                        if patron in synset.name() and count == 0:
                            count = 1
                            for l in synset.lemmas(lang='eus'):
                                synonyms.append(l.name())

                            try:
                                synonyms.remove(entry.text.lower())
                            except:
                                pass
                            try:
                                synonyms.remove(entry.lemma)
                            except:
                                pass

                            definition = synset.definition()
                            offset = synset.offset()
                            #ejemplos
                            examples = synset.examples()
                            #traduccion es
                            traduccion_es=synset.lemmas('spa')
                            es_list=[]
                            for l in traduccion_es:
                                es_list.append(l.name())
                            #traduccion eu
                            traduccion_eu=synset.lemmas('eus') 
                            eu_list=[]
                            for l in traduccion_eu:
                                eu_list.append(l.name())
                            #url imagen
                            if entry.upos=='NOUN':
                                #deia wikidata
                                url=wikidata_offset2url(offset)
                                if url == '':
                                    url=wikipedia_offset2url(entry.text)

        if not set_is_empty(set(synonyms)):
            nueve=entry.text+":"+str(set(synonyms))
        else:
            nueve="_"
        
        print(str(entry.index)+"\t"+entry.text+"\t"+entry.lemma+"\t"+entry.upos+"\t"+entry.xpos+"\t"+entry.feats+"\t"+str(entry.governor)+"\t"+str(entry.dependency_relation)+"\t"+definition+"\t"+nueve+"\t"+str(url)+"\t"+str(offset))
        estfile.write("%s" % str(entry.index)+"\t"+entry.text+"\t"+entry.lemma+"\t"+entry.upos+"\t"+entry.xpos+"\t"+entry.feats+"\t"+str(entry.governor)+"\t"+str(entry.dependency_relation[:2])+"\t"+definition+"\t"+nueve+"\t"+str(url)+"\t"+str(offset))
        estfile.write("\n")
    estfile.write("\n")
estfile.close()

[nltk_data] Downloading package wordnet to /home/kepa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw to /home/kepa/nltk_data...
[nltk_data]   Package omw is already up-to-date!


Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/home/kepa/eu/eu_bdt_models/eu_bdt_tokenizer.pt', 'lang': 'eu', 'shorthand': 'eu_bdt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/home/kepa/eu/eu_bdt_models/eu_bdt_tagger.pt', 'pretrain_path': '/home/kepa/eu/eu_bdt_models/eu_bdt.pretrain.pt', 'lang': 'eu', 'shorthand': 'eu_bdt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/home/kepa/eu/eu_bdt_models/eu_bdt_lemmatizer.pt', 'lang': 'eu', 'shorthand': 'eu_bdt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/home/kepa/eu/eu_bdt_models/eu_bdt_parser.pt', 'pretrain_path': '/home/kepa/eu/eu_bdt_models/eu_bdt.pretrain.pt', 'lang': 'eu', 'shorthand': 'eu_bdt', 'mode': 'predict'}
Done loading processors!
---


/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:14: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


1	Fruta	Fruta	NOUN	_	Case=Abs|Definite=Def	0	root	fruit cooked in sugar syrup and encrusted with a sugar crystals	Fruta:{'fruta_konfitatu'}	https://upload.wikimedia.org/wikipedia/commons/b/b0/Ata_Sugar-apple_Pinha_Fruta_do_conde.JPG	7600506
2	izozkia	izozki	NOUN	_	Animacy=Inan|Case=Abs|Definite=Def|Number=Sing	3	obj	a frozen dessert with fruit flavoring (especially one containing no milk)	_		7614198
3	egiteko	egin	VERB	_	Case=Loc	4	advcl	enter or assume a certain state or condition	egiteko:{'bihurtu', 'iz.-egin', 'izond.-tu', 'bilakatu'}		149583
4	osagaiak	osagai	NOUN	_	Animacy=Inan|Case=Abs|Definite=Def|Number=Plur	1	nsubj	a component of a mixture or compound	_		3570709
5	:	:	PUNCT	_	_	1	punct		_		0
1	kilo	kilo	NOUN	_	_	20	nsubj	the staff on which wool or flax is wound before spinning	kilo:{'goru', 'linai'}	https://upload.wikimedia.org/wikipedia/commons/9/99/Wiktionary-logo-en-v2.svg	3212406
2	erdi	erdi	NOUN	_	_	1	det	the property of being central	_	https://upload.wikimedia.org/wikipe

/media/datos/Dropbox/ikerkuntza/metrix-env/lib/python3.6/site-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /media/datos/Dropbox/ikerkuntza/metrix-env/lib/python3.6/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


11	azukre	azukre	NOUN	_	_	7	conj	a white crystalline carbohydrate used as a sweetener and preservative	_		7859284
12	,	,	PUNCT	_	_	14	punct		_		0
13	3	3	NUM	_	NumType=Card	14	nummod		_		0
14	arrautza	arrautza	NOUN	_	_	11	nmod	animal reproductive body consisting of an ovum or embryo together with nutritive and protective envelopes; especially the thin-shelled reproductive body laid by e.g. female birds	_		1460457
15	zuringo	zurin	ADJ	_	_	14	amod		_		0
16	eta	eta	CCONJ	_	_	19	cc		_		0
17	limoi	limoi	NOUN	_	_	19	nmod	yellow oval fruit with juicy acidic flesh	_	https://upload.wikimedia.org/wikipedia/commons/e/e4/Lemon.jpg	7749582
18	baten	bat	NUM	_	NumType=Card	17	nummod		_		0
19	zukua	zuku	NOUN	_	Animacy=Inan|Case=Abs|Definite=Def|Number=Sing	15	conj	the liquid part that can be extracted from plant or animal tissue by squeezing or cooking	zukua:{'ur'}	https://upload.wikimedia.org/wikipedia/commons/b/b6/Catatumbolightning.jpg	7923748
20	.	.	PUNCT	_	_	28	punct		_		0
21	Lehenik	lehenik	ADV	_

For english:

In [20]:
# coding=utf-8
import nltk
import sys
import re
import os

from wordcloud import WordCloud
from wordfreq import zipf_frequency

#################WORDNET##########################
#Instalar 'wordnet' 
nltk.download('wordnet')
#Add multilingual wordnet
nltk.download('omw')
from nltk.corpus import wordnet as wn
def set_is_empty(some_set):
    return some_set == set()
##################################################

from nltk.wsd import lesk
###############ImageNet###########################
#from bs4 import BeautifulSoup
#import requests
#def imagenet_offset2url(offset):
#    soffset=str(offset).zfill(8)
#    query="http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n"+str(soffset)
#    page = requests.get(query)#ship synset
    # BeautifulSoup is an HTML parsing library
    #puts the content of the website into the soup variable, each url on a different line
#    soup = BeautifulSoup(page.content, 'html.parser')
#    str_soup=str(soup)#convert soup to string so it can be split
#    split_urls=str_soup.split('\r\n')#split so each url is a different possition on a list
#    return(split_urls[0])
###############wikipedia############################
import wikipedia
def wikipedia_offset2url(text):
#WIKIPEDIA
    url= ''
    ext = ''
    svg = False
    i = 0
    while (svg==False):
        try:
            #First of all wikipedia.page() will store all the relevant informations 
            #from the requested page in the variable imagepage. 
            imagepage=wikipedia.page(text)
            #Existen muchas funciones. imagepage.url devuelve la url, imagepage.url.content el contenido, title el título....
            #imagepage.images[0] will return the URL of the image that is present at index 0. 
            #If you want to fetch another image use index as 1, 2, 3, etc, according to images present in the page.
            url = imagepage.images[i]
            #print(url)
            name, ext =(url.split('/')[-1].split('.'))
            if ext=='.svg':
                  i = i+1
            else:
                  svg = True
        except:
            url= ''
            svg = True
    return url
                #si no desambiguamos
##########Wikidata###################################
#Wikidata query service -> code in python
#pip install wikidata
#https://pypi.org/project/Wikidata/
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/
import wikidata
from SPARQLWrapper import SPARQLWrapper, JSON
from wikidata.client import Client
def get_results(endpoint_url, query):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()
def wikidata_offset2url(offset):
    #WIKIDATA
    endpoint_url = "https://query.wikidata.org/sparql"
    soffset=str(offset).zfill(8)
    query="Select * where {?item wdt:P2888 <http://wordnet-rdf.princeton.edu/wn30/"+soffset+"-n> }"
    try:
        results = get_results(endpoint_url, query)
        ema=results["results"]["bindings"]
        item=ema[0].get('item')
        value=item.get('value')
        head, tail = value.split("http://www.wikidata.org/entity/")
        #for result in results["results"]["bindings"]:
          #    print(result)
        client = Client()
        entity = client.get(tail, load=True)
        image_prop = client.get('P18')
        image = entity[image_prop]
        #image-><wikidata.commonsmedia.File 'File:KBS "The Producers" press conference, 11 May 2015 10.jpg'>
        #image.image_resolution
        url=image.image_url
    except:
        url=''
    return url
#####################################################
#Este método devuelve true en caso de que la palabra pasada como parámetro sea verbo. Para que una palabra sea 
#verbo se tiene que cumplir que sea VERB o que sea AUX y que su padre NO sea VERB.

####analizador sintactico#######################
import stanfordnlp
stanford = stanfordnlp.Pipeline()
#stanfordnlp.download("en")

####input file##################
input="receta.txt" #sys.argv[1]

####nivel de dificultad#########
difficult = 5 #int(sys.argv[2])

####ouput files#################
#estadisticos
estadisticaoutput=input+".out.csv"

#Write all the information in the file
estfile = open(estadisticaoutput, "w")


###############Tratamiento de texto###############################################
#quitar todos los retornos \n si contiene
text = open(input).read().replace('\n', '')
#remove text inside parentheses
#text = re.sub(r'\([^)]*\)', '', text)
#separa , . ! ( ) ? ; del texto con espacios, teniendo en cuenta que los no son numeros en el caso de , y . 
text = re.sub(r'[.]+(?![0-9])', r' . ', text)
text = re.sub(r'[,]+(?![0-9])', r' , ', text)
text = re.sub(r"!", " ! ", text)
text = re.sub(r"\(", " ( ", text)
text = re.sub(r"\)", " ) ", text)
text = re.sub(r"\?", " ? ", text)
text = re.sub(r";", " ; ", text)
#sustituye 2 espacios seguidos por 1
text = re.sub(r"\s{2,}", " ", text)
##############################################################################################
#Generating a word cloud with no optional parameters based on the above string:
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)
#stopwords.add("every")
#stopwords.add("will")
#stopwords={'to', 'of', 'us'}
#Generating a word cloud with no optional parameters based on the above string:
# #This is because the wordcloud module ignores stopwords by default. Refer to Part 1 of the NLTK tutorial if the concept of stopwords is new to you.If we wish, we can specify our own set of stopwords, instead of the stopwords provided by default.
# #Con relative_scaling = 0, solo se consideran los rangos de las palabras. Si modificamos esto a relative_scaling = 1.0, entonces una palabra que aparece dos veces más frecuentemente aparecerá dos veces el tamaño. Por defecto, relative_scaling = 0.5.
# wordcloud = WordCloud(relative_scaling=1.0, stopwords={'to', 'of'}).generate(text)
wordcloud = WordCloud(relative_scaling=1.0,stopwords=stopwords).generate(text)

#Finally, use matplotlib to render the word cloud:
#plot_wordcloud(wordcloud)
wordcloudfilename=input+".png"
#wordcloudfilename="resume.png"
wordcloud.to_file(wordcloudfilename) 

###################analizador morfosintactico#################################################
palabras_diferentes = []
doc=stanford(text)
for sent in doc.sentences:
    #Por cada sentencia
    for entry in sent.words:
        definition = ''
        synonyms = []
        offset = 0
        nueve = '_' 
        url = ''              
        #Por cada palabra
        if entry.text.isalpha() and (entry.upos == 'NOUN' or entry.upos == 'VERB') and entry.text.lower() not in palabras_diferentes:
            #Si la palabra es un content word
            palabras_diferentes.append(entry.text.lower())
            wordfrequency = zipf_frequency(entry.text.lower(), 'en')
            #lemafrequency = zipf_frequency(entry.lemma, 'en')
            #Alto<=3 (1 por millon)
            #Medio<=4 (10 por millon) 
            #Bajo<=5 (100 por millon)
            if wordfrequency <= int(difficult):
                lema=entry.lemma
                synset_ids = wn.synsets(lema)
                if entry.upos=='NOUN':
                    patron='n'
                if entry.upos=='VERB':
                    patron='v'
                #intentamos obtener la synset desambiguada, y la def,exam,sin and img-url sobre esa synset
                try:
                    synset_desambiguado = lesk(text.split(), entry.text, pos=patron, synsets=synset_ids)
                    #definicion
                    definition = synset_desambiguado.definition()
                    #ejemplos
                    examples = synset_desambiguado.examples()
                    #offset
                    offset = synset_desambiguado.offset()
                    #traduccion es
                    traduccion_es=answer2.lemmas('spa')
                    es_list=[]
                    for l in traduccion_es:
                        es_list.append(l.name())
                    #traduccion eu
                    traduccion_eu=answer2.lemmas('eus') 
                    eu_list=[]
                    for l in traduccion_eu:
                        eu_list.append(l.name())
                    #sinonimos
                    for l in synset_desambiguado.lemma_names():
                        synonyms.append(l.lower())
                    try:
                        synonyms.remove(entry.text.lower())
                    except:
                        pass
                    try:
                        synonyms.remove(entry.lemma)
                    except:
                        pass

                    #url imagen
                    if entry.upos=='NOUN':
                        #deia wikidata
                        url=wikidata_offset2url(offset)
                        if url == '':
                            url=wikipedia_offset2url(entry.text)
                #si no desambiguamos
                except:
                    count = 0
                    for synset in synset_ids:
                        if entry.upos=='NOUN':
                            patron='.n.'
                        if entry.upos=='VERB':
                            patron='.v.'

                        if patron in synset.name() and count == 0:
                            count = 1
                            for l in synset.lemmas():
                                synonyms.append(l.name())

                            try:
                                synonyms.remove(entry.text.lower())
                            except:
                                pass
                            try:
                                synonyms.remove(entry.lemma)
                            except:
                                pass

                            definition = synset.definition()
                            offset = synset.offset()
                            #ejemplos
                            examples = synset.examples()
                            #traduccion es
                            traduccion_es=synset.lemmas('spa')
                            es_list=[]
                            for l in traduccion_es:
                                es_list.append(l.name())
                            #traduccion eu
                            traduccion_eu=synset.lemmas('eus') 
                            eu_list=[]
                            for l in traduccion_eu:
                                eu_list.append(l.name())
                            #url imagen
                            if entry.upos=='NOUN':
                                #deia wikidata
                                url=wikidata_offset2url(offset)
                                if url == '':
                                    url=wikipedia_offset2url(entry.text)

        if not set_is_empty(set(synonyms)):
            nueve=entry.text+":"+str(set(synonyms))
        else:
            nueve="_"
        
        print(str(entry.index)+"\t"+entry.text+"\t"+entry.lemma+"\t"+entry.upos+"\t"+entry.xpos+"\t"+entry.feats+"\t"+str(entry.governor)+"\t"+str(entry.dependency_relation)+"\t"+definition+"\t"+nueve+"\t"+str(url)+"\t"+str(offset))
        estfile.write("%s" % str(entry.index)+"\t"+entry.text+"\t"+entry.lemma+"\t"+entry.upos+"\t"+entry.xpos+"\t"+entry.feats+"\t"+str(entry.governor)+"\t"+str(entry.dependency_relation[:2])+"\t"+definition+"\t"+nueve+"\t"+str(url)+"\t"+str(offset))
        estfile.write("\n")
    estfile.write("\n")
estfile.close()

[nltk_data] Downloading package wordnet to /home/kepa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw to /home/kepa/nltk_data...
[nltk_data]   Package omw is already up-to-date!


Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/home/kepa/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/home/kepa/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/home/kepa/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/home/kepa/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/home/kepa/stanfordnlp_resources/en_ewt_models/en_ewt_parser.pt', 'pretrain_path': '/home/kepa/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain

/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:14: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


1	Fry	fry	VERB	VB	Mood=Imp|VerbForm=Fin	0	root	be excessively hot	_		2738241
2	the	the	DET	DT	Definite=Def|PronType=Art	3	det		_		0
3	garlic	garlic	NOUN	NN	Number=Sing	1	obj	bulbous herb of southern Europe widely naturalized; bulb breaks up into separate strong-flavored cloves	garlic:{'Allium_sativum'}	https://upload.wikimedia.org/wikipedia/commons/8/8b/All_Garlic_Ail_Ajo.jpg	12434775
4	and	and	CCONJ	CC	_	6	cc		_		0
5	the	the	DET	DT	Definite=Def|PronType=Art	6	det		_		0
6	chilly	chilly	ADJ	JJ	Degree=Pos	3	conj		_		0
7	in	in	ADP	IN	_	9	case		_		0
8	the	the	DET	DT	Definite=Def|PronType=Art	9	det		_		0
9	oil	oil	NOUN	NN	Number=Sing	6	obl		_		0
10	on	on	ADP	IN	_	13	case		_		0
11	a	a	DET	DT	Definite=Ind|PronType=Art	13	det		_		0
12	moderate	moderate	ADJ	JJ	Degree=Pos	13	amod		_		0
13	heat	heat	NOUN	NN	Number=Sing	1	obl	a form of energy that is transferred by a difference in temperature	heat:{'heat_energy'}	https://upload.wikimedia.org/wikipedia/commons/d/da/171879main_LimbFlareJan12_lg.jpg	

In [ ]:
#convertir svg a png
import cairo
import rsvg
from xml.dom import minidom


def convert_svg_to_png(svg_file, output_file):
    # Get the svg files content
    with open(svg_file) as f:
        svg_data = f.read()

    # Get the width / height inside of the SVG
    doc = minidom.parse(svg_file)
    width = int([path.getAttribute('width') for path
                 in doc.getElementsByTagName('svg')][0])
    height = int([path.getAttribute('height') for path
                  in doc.getElementsByTagName('svg')][0])
    doc.unlink()

    # create the png
    img = cairo.ImageSurface(cairo.FORMAT_ARGB32, width, height)
    ctx = cairo.Context(img)
    handler = rsvg.Handle(None, str(svg_data))
    handler.render_cairo(ctx)
    img.write_to_png(output_file)

if __name__ == '__main__':
    from argparse import ArgumentParser

    parser = ArgumentParser()

    parser.add_argument("-f", "--file", dest="svg_file",
                        help="SVG input file", metavar="FILE")
    parser.add_argument("-o", "--output", dest="output", default="svg.png",
                        help="PNG output file", metavar="FILE")
    args = parser.parse_args()


In [23]:
#wikipedia: pip install wikipedia
#url imagen
##################IMAGES##################################################################
import wikipedia

#set_lang() method is used to set the language that you want to set.It takes an argument that is prefix of the language like for arabic prefix is ar and so on.
#If you want to know about the prefix of the different languages then refer this link but make sure that the Wikipedia should have that article in the language you want.
#Here i have set the language arabic and article is facebook.It will give the summary of facebook in arabic language that’s so amazing.
#wikipedia.set_lang("eus")
#wikipedia.summary("wolf")

upos='NOUN'
text='dog'
ext =''
svg = False
name=''
i = 0
if upos=='NOUN':
     while (svg==False):
        try:
            #First of all wikipedia.page() will store all the relevant informations 
            #from the requested page in the variable imagepage. 
            imagepage=wikipedia.page(text)
            #Existen muchas funciones. imagepage.url devuelve la url, imagepage.url.content el contenido, title el título....
            #imagepage.images[0] will return the URL of the image that is present at index 0. 
            #If you want to fetch another image use index as 1, 2, 3, etc, according to images present in the page.
            url = imagepage.images[i]
            name, ext =(url.split('/')[-1].split('.'))
            if ext=='.svg':
                 i = i+1
            else:
                 svg = True
        except:
            svg = True
print(str(url))
print("name:"+name)
print("ext:"+ext)
#segunda opcion:
url2="http://en.wikipedia.org/w/api.php?action=query&titles="+text+"&generator=images&gimlimit=10&prop=imageinfo&iiprop=url|dimensions|mime&format=json" 
#Nos podemos bajar el fichero json y obtener la imagen que mejor nos convenga.
#Solo hay que cambiar la parte de titles
print(str(url2))


https://upload.wikimedia.org/wikipedia/commons/d/df/2018_06_TropicalIslands_IMG_2170.jpg
name:
ext:
http://en.wikipedia.org/w/api.php?action=query&titles=dog&generator=images&gimlimit=10&prop=imageinfo&iiprop=url|dimensions|mime&format=json


\item Wikidata
El repositorio Wikidata consta principalmente de elementos, cada uno conteniendo una etiqueta, una descripción y cualquier número de alias. Los elementos son identificados de manera única por un prefijo Q seguido por un número, como Douglas Adams (Q42). Las declaraciones describen características detalladas de cada elemento y constan de una propiedad y un valor. Las propiedades en Wikidata tienen un prefijo P seguido de un número, como con educated at (P69). 
Por ejemplo:
Elementos 	Propiedad 	Valor
Q42 	P69 	Q691283
Douglas Adams 	educated at 	St John's College 

Mediante la propiedad P2888 se puede unir los elementos de Wikidata con WordNet (mediante la "exact match(P2888)" usado para conectar dos conceptos, indica un alto grado de confianza de que el concepto puede ser usado de forma intercambiable)
	
Elementos 	Propiedad 	Valor
Q322787 	P69 	Q691283
mashed potato (exact match)P2888 http://wordnet-rdf.princeton.edu/wn30/07711569-n

También puedo acceder a las imagenes desde wikidata:
%image of relevant illustration of the subject; if available, use more specific properties (sample: coat of arms image, locator map, flag image, signature image, logo image, collage image); only images which exist on Wikimedia Commons are acceptable
Elementos 	Propiedad 	Valor
Q322787 	 image (P18)  Aligot.jpg
También se puede acceder a BabelNet (\url{http://babelnet.org}
Elementos 	Propiedad 	Valor
https://www.wikidata.org/wiki/Q3490685 	  BabelNet ID (P2581)   https://babelnet.org/synset?word=bn:00039711n

¿Podriamos acceder directamente desde WordNet a BabelNet?
No, ya que no utiliza los mismos identificadores.

¿Puedo acceder desde wordnet a wikidata images?
Si usando Wikidata Query Service (WDQS) SPARQL query:
https://query.wikidata.org/

In [25]:
#Wikidata
#Wikidata query service -> code in python 
#https://pypi.org/project/Wikidata/
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import wikidata
from SPARQLWrapper import SPARQLWrapper, JSON
from wikidata.client import Client


#url="https://query.wikidata.org/bigdata/namespace/wdq/sparql?query={"+consulta

#query = """Select * where {?item wdt:P2888 <http://wordnet-rdf.princeton.edu/wn30/07711569-n> } """
def get_results(endpoint_url, query):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


#offset=7711569
endpoint_url = "https://query.wikidata.org/sparql"
soffset=str(offset).zfill(8)
query="Select * where {?item wdt:P2888 <http://wordnet-rdf.princeton.edu/wn30/"+soffset+"-n> }"
results = get_results(endpoint_url, query)
ema=results["results"]["bindings"]
item=ema[0].get('item')
value=item.get('value')
head, tail = value.split("http://www.wikidata.org/entity/")
print(tail)
#for result in results["results"]["bindings"]:
#    print(result)
client = Client()
entity = client.get(tail, load=True)
print(entity.description)
image_prop = client.get('P18')
image = entity[image_prop]
#image-><wikidata.commonsmedia.File 'File:KBS "The Producers" press conference, 11 May 2015 10.jpg'>
image.image_resolution
print(str(image.image_url))

Q322787
potato dish
https://upload.wikimedia.org/wikipedia/commons/7/71/Aligot.jpg


In [6]:
#ImageNet
from bs4 import BeautifulSoup
import requests
def imagenet_offset2url(offset):
    soffset=str(offset).zfill(8)
    query="http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n"+str(soffset)
    page = requests.get(query)#ship synset
    # BeautifulSoup is an HTML parsing library
    #puts the content of the website into the soup variable, each url on a different line
    soup = BeautifulSoup(page.content, 'html.parser')
    str_soup=str(soup)#convert soup to string so it can be split
    split_urls=str_soup.split('\r\n')#split so each url is a different possition on a list
    return(split_urls[0])
imagenet_offset2url(4194289) #ship synset

SyntaxError: invalid syntax (<ipython-input-6-3dca22d3506b>, line 6)

In [3]:
import nltk
import sys
import re


####input file##################
input="receta.txt" #sys.argv[1]
####ouput files#################
#estadisticos
estadisticaoutput=input+".out.csv"

#Write all the information in the file
estfile = open(estadisticaoutput, "w")


###############Tratamiento de texto###############################################
#quitar todos los retornos \n si contiene
text = open(input).read().replace('\n', '')
#remove text inside parentheses
#text = re.sub(r'\([^)]*\)', '', text)
#separa , . ! ( ) ? ; del texto con espacios, teniendo en cuenta que los no son numeros en el caso de , y . 
text = re.sub(r'[.]+(?![0-9])', r' . ', text)
text = re.sub(r'[,]+(?![0-9])', r' , ', text)
text = re.sub(r"!", " ! ", text)
text = re.sub(r"\(", " ( ", text)
text = re.sub(r"\)", " ) ", text)
text = re.sub(r"\?", " ? ", text)
text = re.sub(r";", " ; ", text)
#sustituye 2 espacios seguidos por 1
text = re.sub(r"\s{2,}", " ", text)
################STOPWORDS##############################################################################
#Generating a word cloud with no optional parameters based on the above string:
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)
def set_is_empty(some_set):
    return some_set == set()
#stopwords.add("every")
#stopwords.add("will")
#stopwords={'to', 'of', 'us'}
##############WORDCLOUD########################################################################
#matplotlib kendu dezakegu 
import matplotlib
from wordcloud import WordCloud
#Generating a word cloud with no optional parameters based on the above string:
# #This is because the wordcloud module ignores stopwords by default. Refer to Part 1 of the NLTK tutorial if the concept of stopwords is new to you.If we wish, we can specify our own set of stopwords, instead of the stopwords provided by default.
# #Con relative_scaling = 0, solo se consideran los rangos de las palabras. Si modificamos esto a relative_scaling = 1.0, entonces una palabra que aparece dos veces más frecuentemente aparecerá dos veces el tamaño. Por defecto, relative_scaling = 0.5.
# wordcloud = WordCloud(relative_scaling=1.0, stopwords={'to', 'of'}).generate(text)
wordcloud = WordCloud(relative_scaling=1.0,stopwords=stopwords).generate(text)
#Finally, use matplotlib to render the word cloud:
#plot_wordcloud(wordcloud)
wordcloudfilename=input+".png"
#wordcloudfilename="resume.png"
wordcloud.to_file(wordcloudfilename) 
###################WORDFREQ#################################################################
from wordfreq import zipf_frequency

##################PARSER#############################################################
import stanfordnlp
#stanfordnlp.download("en")
stanford=stanfordnlp.Pipeline()

##################WORDNET############################################################ 
#Instalar 'wordnet' 
nltk.download('wordnet')
#Add multilingual wordnet
nltk.download('omw')
from nltk.corpus import wordnet as wn
def set_is_empty(some_set):
    return some_set == set()
###################WSD########################################################################
#lesk erabiltzeko
from nltk.wsd import lesk
#simple_lesk erabiltzeko
#nltk.download('averaged_perceptron_tagger')
#from pywsd.lesk import simple_lesk

##################IMAGES##################################################################
import wikipedia
###################MAIN#################################################
palabras_raras = []
palabras_diferentes = []
doc=stanford(text)
for sent in doc.sentences:
	#Por cada sentencia
	sentence=""
	for entry in sent.words:
		sentence=sentence+' '+entry.text
	print(sentence)
	print(sentence.split())
	for entry in sent.words:
		nueve='_'
		definicion= ''
		synonyms = []
		offset=0
		url=''
		#Por cada palabra
		if entry.text.isalpha() and (entry.upos == 'NOUN' or entry.upos == 'VERB') and entry.text.lower() not in palabras_diferentes:
			#Si la palabra es un content word
			palabras_diferentes.append(entry.text.lower())
			wordfrequency = zipf_frequency(entry.text.lower(), 'en')
			#lemafrequency = zipf_frequency(entry.lemma, 'en')
			if wordfrequency <= 8:
				#Si es rara
				palabras_raras.append(entry.text)
				#Todos los synsets del lema 
				synset_ids=wn.synsets(entry.lemma)
				if entry.upos=='NOUN':
					patron='n'
					#synset_ids=wn.synsets(entry.lemma, pos=wn.NOUN)
				if entry.upos=='VERB':
					patron='v'
					#synset_ids=wn.synsets(entry.lemma, pos=wn.VERB)
				#print(entry.lemma+":"+str(synset_ids))
				#Desambiguador
				#print('\n',str(entry.upos))                    
				#answer1=lesk(sentence,entry.text)
				#print('\n',str(answer1))
				#print('\t',answer1.definition())
				#answer2 = simple_lesk(sentence, entry.text, pos=patron)
				#print('\n',str(answer2))
				#print('\t',answer2.definition())
				#definicion=answer2.definition()
				#print('\t',answer2.lemma_names())
				#print('\t',answer2.offset())
				#INTENTO DESAMBIGUAR CON LESK            
				try:
					answer2 = lesk(text.split(), entry.text, pos=patron, synsets=synset_ids)
					definicion=answer2.definition()
					print("Definición:")
					print(definicion)
					ejemplos=answer2.examples()
					print("Ejemplos:")
					print(ejemplos)
					print("Traduccion:")                    
					traduccion_es=answer2.lemmas('spa') #.split('.')[3]
					es_list=[]
					for l in traduccion_es:
						es_list.append(l.name())
					print(es_list)                    
					traduccion_eu=answer2.lemmas('eus') #.split('.')[3]
					eu_list=[]
					for l in traduccion_eu:
						eu_list.append(l.name())
					print(eu_list) 
#					print("Lemmas_names:")                    
#					print(answer2.lemmas_names()) 
#\item De ImageNet \cite{imagenet} que es una colección de imágenes etiquetadas contra WordNet 3.0 y descritas en http://image-net.org. Contiene 14197122 imágenes y 21841 synsets indexados.Por ejemplo:platano
#\url{http://image-net.org/explore.php?wnid=n07753592}
#\url{http://wordnet-rdf.princeton.edu/wn30/07753592-n}

					if entry.upos=='NOUN':
						url="http://image-net.org/explore.php?wnid=n"+str(answer2.offset()).zfill(8)
						url="http://image-net.org/synset?wnid=n"+str(answer2.offset()).zfill(8)
					if entry.upos=='VERB':
						url="http://image-net.org/explore.php?wnid=v"+str(answer2.offset()).zfill(8)
						url="http://image-net.org/synset?wnid=v"+str(answer2.offset()).zfill(8) 
					#offset="http://www.image-net.org/synset?wnid="+str(answer2.offset())
					#offset="015201505" #str(answer2.offset())                          
					#sparql="SELECT * WHERE {?item wdt:P2888 <http://wordnet-rdf.princeton.edu/wn30/"+offset+"-n>}"
					#url="https://query.wikidata.org/bigdata/namespace/wdq/sparql?query="+sparql
					#try:
						#name=re.sub(r"\_", " ", answer2.name().split(".")[0])
					#	offset=wikipedia.page(entry.text).images[0]
					#except:
					#	pass                        
					#si tiene sinonimos
					for l in answer2.lemma_names():
						synonyms.append(l.lower())
						#print(l.lower())
					#print(entry.text.lower())
					try:
						synonyms.remove(entry.text.lower())
					except:
						pass
					try:
						synonyms.remove(entry.lemma)
					except:
						pass
				#COJO EL PRIMER SYNSET QUE CUMPLA LA RESTRICCIÓN POS
				except:
					count=0
					for synset in synset_ids: # [Synset('libra.n.03'), Synset('libra.n.02'), Synset('pound.n.01')]
						if entry.upos=='NOUN':
							patron='.n.'
						if entry.upos=='VERB':
							patron='.v.'
						if patron in synset.name() and count==0:
							count=1
							#print ("synset.name():")                 
							#print (synset.name())   #pound.n.01
							#sinonimoak bilatu
							#print (str(synset.lemmas(lang='eus')))
							for l in synset.lemmas(): #[Lemma('pound.n.01.libera'), Lemma('pound.n.01.libra')]
								synonyms.append(l.name('spa'))
								#print ("l.name():") #libera, libra              
								#print (l.name())
							try:
								synonyms.remove(entry.text.lower())
							except:
								pass
							try:
								synonyms.remove(entry.lemma)
							except:
								pass
							definicion=synset.definition()
							if entry.upos=='NOUN':
								url="http://image-net.org/explore.php?wnid=n"+str(synset.offset()).zfill(8)
								url="http://image-net.org/synset?wnid=n"+str(synset.offset()).zfill(8)
								url="http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=" + str(synset.offset()).zfill(8)
							if entry.upos=='VERB':
								url="http://image-net.org/explore.php?wnid=v"+str(synset.offset()).zfill(8)
								url="http://image-net.org/synset?wnid=v"+str(synset.offset()).zfill(8)
                            #str(1).zfill(3);
							#offset=str(synset.offset())
							#offset="015201505" #str(answer2.offset())                          
							#sparql="SELECT * WHERE {?item wdt:P2888 <http://wordnet-rdf.princeton.edu/wn30/"+offset+"-n>}"
							#url="https://query.wikidata.org/bigdata/namespace/wdq/sparql?query="+sparql
							#offset="http://www.image-net.org/synset?wnid="+str(synset.offset())
							#try:
								#name=re.sub(r"\_", " ", synset.name().split(".")[0])
								#offset=wikipedia.page(name).images[0]
							#except:
								#pass
		if not set_is_empty(set(synonyms)):
			nueve=entry.text+":"+str(set(synonyms))
		else:
			nueve="_"
		print(entry.text+"\t"+entry.lemma+"\t"+entry.upos+"\t"+entry.xpos+"\t"+entry.feats+"\t"+str(entry.governor)+"\t"+str(entry.dependency_relation)+"\t"+definicion+"\t"+nueve+"\t"+str(url))
		estfile.write("%s" % entry.text+"\t"+entry.lemma+"\t"+entry.upos+"\t"+entry.xpos+"\t"+entry.feats+"\t"+str(entry.governor)+"\t"+str(entry.dependency_relation[:2])+"\t"+definicion+"\t"+nueve+"\t"+str(url))
		estfile.write("\n")
estfile.close()

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/home/kepa/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/home/kepa/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/home/kepa/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/home/kepa/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/home/kepa/stanfordnlp_resources/en_ewt_models/en_ewt_parser.pt', 'pretrain_path': '/home/kepa/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain

[nltk_data] Downloading package wordnet to /home/kepa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw to /home/kepa/nltk_data...
[nltk_data]   Package omw is already up-to-date!
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:14: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


For Basque:
Understanding WordNet: Instalar 'wordnet' baja muchos ficheros a /home/kepa/nltk_data/corpora/wordnet nltk.download('wordnet'). To add multilingual wordnet baja un fichero para el euskara a /home/kepa/nltk_data/corpora/omw/eus/wn-data-eus.tab nltk.download('omw') euskarazko lema="Libra" /home/kepa/nltk_data/corpora/omw/eus/wn-data-eus.tab fitxategian:
# Multilingual Central Repository       eus     http://adimen.si.ehu.es/web/MCR/        CC BY 3.0
00001740-n      lemma   entitate
00001740-n      lemma   izaki
00001740-n      lemma   sorkari
00001740-v      lemma   arnasa hartu
00001740-v      lemma   arnastu
00001740-v      lemma   hats hartu
00002137-n      lemma   abstrakzio
00002684-n      lemma   gauza
00002684-n      lemma   gauzaki
00002684-n      lemma   objektu


3 sarrera ditu: 08687150-n, 09339109-n, 1372009696-n

wn.

http://adimen.si.ehu.es/cgi-bin/wei/public/wei.consult.perl

Ikusi:
http://www.nltk.org/howto/wordnet.html

https://github.com/nltk/nltk_data/issues/43

https://codeday.me/es/qa/20190503/633497.html

Aupa Kepa! hemen dago dena:

http://ixa2.si.ehu.es/ukb/

Euskarako Wordnet grafo eta hitzegiak ere hor daude.

UKBren kodea hemen:

https://github.com/asoroa/ukb

Katxarreatzen hasi eta edozein duda bihar han egongo naiz
UKB: Graph Base WSD,[53] a collection of programs for performing graph-based Word Sense Disambiguation and lexical similarity/relatedness using a pre-existing Lexical Knowledge Base[54]
Ejemplo de uso: Basajaun
cd /tartalo03/users/qtleap/pipeline/EU
cat proba_fitxategiak/proba4.txt | sh ixa-pipe-pos-eu/run.sh | sh  ixa-pipe-wsd-ukb/run.sh


Bert
https://github.com/uhh-lt/bert-sense

In [4]:
import nltk
import sys
import re


####input file##################
input="errezeta.txt" #sys.argv[1]
####ouput files#################
#estadisticos
estadisticaoutput=input+".out.csv"

#Write all the information in the file
estfile = open(estadisticaoutput, "w")


###############Tratamiento de texto###############################################
#quitar todos los retornos \n si contiene
text = open(input).read().replace('\n', '')
#remove text inside parentheses
#text = re.sub(r'\([^)]*\)', '', text)
#separa , . ! ( ) ? ; del texto con espacios, teniendo en cuenta que los no son numeros en el caso de , y . 
text = re.sub(r"\_", " ", text)
text = re.sub(r"\-", " ", text)
text = re.sub(r'[.]+(?![0-9])', r' . ', text)
text = re.sub(r'[,]+(?![0-9])', r' , ', text)
text = re.sub(r"!", " ! ", text)
text = re.sub(r"\(", " ( ", text)
text = re.sub(r"\)", " ) ", text)
text = re.sub(r"\?", " ? ", text)
text = re.sub(r";", " ; ", text)
#sustituye 2 espacios seguidos por 1
text = re.sub(r"\s{2,}", " ", text)
##############STOPWORDS################################################################################
#Generating a word cloud with no optional parameters based on the above string:
#from wordcloud import STOPWORDS
#stopwords = set(STOPWORDS)
#from nltk.corpus import stopwords
#stopwords = set(stopwords.words("english"))
#words = [w for w in words if not w in stopwords]
#stopwords=set()
#with open("stopwords_formak.txt") as file:
#    for line in file:
#        line = line.strip() #preprocess line
#        stopwords.add(line)
#print (stopwords)
stopwords= set(line.strip() for line in open('stopwords_formaketakonektoreak.txt'))
stopwords.add("gero")
stopwords.add("ondo")
stopwords.add("ondoren")
stopwords.add("ordu")
stopwords.add("azkenik")
stopwords.add("jarraian")
stopwords.add("igaro")
stopwords.add("berriro")
stopwords.add("lehenik")
#Gehitu konektoreak

#stopwords={'to', 'of', 'us'}
################WORDCLOUD################################################
import matplotlib
from wordcloud import WordCloud
#Generating a word cloud with no optional parameters based on the above string:
# #This is because the wordcloud module ignores stopwords by default. Refer to Part 1 of the NLTK tutorial if the concept of stopwords is new to you.If we wish, we can specify our own set of stopwords, instead of the stopwords provided by default.
# #Con relative_scaling = 0, solo se consideran los rangos de las palabras. Si modificamos esto a relative_scaling = 1.0, entonces una palabra que aparece dos veces más frecuentemente aparecerá dos veces el tamaño. Por defecto, relative_scaling = 0.5.
# wordcloud = WordCloud(relative_scaling=1.0, stopwords={'to', 'of'}).generate(text)
wordcloud = WordCloud(relative_scaling=1.0,stopwords=stopwords).generate(text)
#Finally, use matplotlib to render the word cloud:
#plot_wordcloud(wordcloud)
wordcloudfilename=input+".png"
#wordcloudfilename="resume.png"
wordcloud.to_file(wordcloudfilename)

###################PARSER#################################################
import stanfordnlp #->/media/datos/Dropbox/ikerkuntza/metrix-env/lib/python3.6/site-packages/stanfordnlp
MODELS_DIR = '/home/kepa/eu'
#stanfordnlp.download('eu', MODELS_DIR) # Download the English models
config = {'processors': 'tokenize,pos,lemma,depparse', # Comma-separated list of processors to use
'lang': 'eu', # Language code for the language to build the Pipeline in
'tokenize_model_path': '/home/kepa/eu/eu_bdt_models/eu_bdt_tokenizer.pt', # Processor-specific arguments are set with keys "{processor_name}_{argument_name}"
'pos_model_path': '/home/kepa/eu/eu_bdt_models/eu_bdt_tagger.pt',
'pos_pretrain_path': '/home/kepa/eu/eu_bdt_models/eu_bdt.pretrain.pt',
'lemma_model_path': '/home/kepa/eu/eu_bdt_models/eu_bdt_lemmatizer.pt',
'depparse_model_path': '/home/kepa/eu/eu_bdt_models/eu_bdt_parser.pt',
'depparse_pretrain_path': '/home/kepa/eu/eu_bdt_models/eu_bdt.pretrain.pt'
}
parser = stanfordnlp.Pipeline(**config)
##################WORDFRECUENCY###################################################
# load a freq file into a dictionary
import codecs
d = {}
#f = codecs.open('compounds.dat', encoding='utf-8')
with codecs.open('LB2014Maiztasunak_zenbakiakKenduta.csv',encoding='utf-8') as f:
    next(f)
    for line in f:
       (val,key) = line.split(",")
       d[key] = val
#for key,val in d.items():
 #   print (key, "=>", val)
    
def zipf_frequency_eu(lemma):
    if d.get(lemma):
        return (int(d.get(lemma)))
    else:
        return 1
################WORDNET###################################################
#Instalar 'wordnet' 
nltk.download('wordnet')
#Add multilingual wordnet
nltk.download('omw')
from nltk.corpus import wordnet as wn
def set_is_empty(some_set):
    return some_set == set()
###################WSD####################################################
#simple_lesk erabiltzeko
#nltk.download('averaged_perceptron_tagger')
#from pywsd.lesk import simple_lesk
from nltk.wsd import lesk
##################IMAGES##################################################################
import wikipedia

#################MAIN#############################################################
palabras_raras = []
palabras_diferentes = []
doc=parser(text)
for sent in doc.sentences:
	#Por cada sentencia
	sentence=""
	for entry in sent.words:
		sentence=sentence+' '+entry.text
	#print(sentence)
	for entry in sent.words:
		nueve="_"
		definicion= ''
		synonyms = []
		offset=0
		#Por cada palabra
		#print(entry.text)
		if entry.text.isalpha() and (entry.upos == 'NOUN' or entry.upos == 'VERB') and entry.lemma.lower() not in palabras_diferentes:
			#Si la palabra es un content word
			palabras_diferentes.append(entry.lemma.lower())
			lemafrequency = zipf_frequency_eu(entry.lemma)
			#0-6 zaila, 7-34 erdikoak , 35 errazak
			if lemafrequency <= 35:
				#Si es rara
				palabras_raras.append(entry.text.lower())
				#Libra sartuz gero, eus/wn-data-eus.tab 3 aukera bilatu:08687150-n, 09339109-n, 13720096-n
				#wn.synsets('dog', pos=wn.VERB)
				synset_ids = wn.synsets(entry.lemma,lang='eus')
				if entry.upos=='NOUN':
					patron='n'
				if entry.upos=='VERB':
					patron='v'
				#INTENTO DESAMBIGUAR CON LESK            
				try:
					answer2 = lesk(text.split(), entry.text, pos=patron, synsets=synset_ids)
					definicion=answer2.definition()
					offset="http://www.image-net.org/synset?wnid="+str(answer2.offset())
					try:
						name=re.sub(r"\_", " ", answer2.name().split(".")[0])
						offset=wikipedia.page(name).images[0]
					except:
						pass                        
					#si tiene sinonimos
					for l in answer2.lemma_names():
						synonyms.append(l.lower())
						#print(l.lower())
					#print(entry.text.lower())
					try:
						synonyms.remove(entry.text.lower())
					except:
						pass
					try:
						synonyms.remove(entry.lemma)
					except:
						pass
				#PRIMER SYNSET FILTRADO POR CATEGORIA
				except:
					count=0
					if entry.upos=='NOUN':
						patron='.n.'
					if entry.upos=='VERB':
						patron='.v.'
					for synset in synset_ids: # [Synset('libra.n.03'), Synset('libra.n.02'), Synset('pound.n.01')]
						if patron in synset.name() and count==0:
							count=1
							#print ("synset.name():")                 
							#print (synset.name())   #pound.n.01
							#sinonimoak bilatu
							#print (str(synset.lemmas(lang='eus')))
							for l in synset.lemmas(lang='eus'): #[Lemma('pound.n.01.libera'), Lemma('pound.n.01.libra')]
								synonyms.append(l.name())
								#print ("l.name():") #libera, libra              
								#print (l.name())
							try:
								synonyms.remove(entry.text)
							except:
								pass
							try:
								synonyms.remove(entry.lemma)
							except:
								pass
							definicion=synset.definition()
							offset="http://www.image-net.org/synset?wnid="+str(synset.offset())
							try:
								name=re.sub(r"\_", " ", synset.name().split(".")[0])
								offset=wikipedia.page(name).images[0]
							except:
								pass
		#print("palabras_raras:")
		#print(palabras_raras)
		if not set_is_empty(set(synonyms)):
			nueve=entry.text+":"+str(set(synonyms))
		else:
			nueve="_"
		print(entry.text+"\t"+entry.lemma+"\t"+entry.upos+"\t"+entry.xpos+"\t"+entry.feats+"\t"+str(entry.governor)+"\t"+str(entry.dependency_relation)+"\t"+definicion+"\t"+nueve+"\t"+str(offset))
		estfile.write("%s" % entry.text+"\t"+entry.lemma+"\t"+entry.upos+"\t"+entry.xpos+"\t"+entry.feats+"\t"+str(entry.governor)+"\t"+str(entry.dependency_relation[:2])+"\t"+definicion+"\t"+nueve+"\t"+str(offset))
		estfile.write("\n")
estfile.close()

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/home/kepa/eu/eu_bdt_models/eu_bdt_tokenizer.pt', 'lang': 'eu', 'shorthand': 'eu_bdt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/home/kepa/eu/eu_bdt_models/eu_bdt_tagger.pt', 'pretrain_path': '/home/kepa/eu/eu_bdt_models/eu_bdt.pretrain.pt', 'lang': 'eu', 'shorthand': 'eu_bdt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/home/kepa/eu/eu_bdt_models/eu_bdt_lemmatizer.pt', 'lang': 'eu', 'shorthand': 'eu_bdt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/home/kepa/eu/eu_bdt_models/eu_bdt_parser.pt', 'pretrain_path': '/home/kepa/eu/eu_bdt_models/eu_bdt.pretrain.pt', 'lang': 'eu', 'shorthand': 'eu_bdt', 'mode': 'predict'}
Done loading processors!
---


[nltk_data] Downloading package wordnet to /home/kepa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw to /home/kepa/nltk_data...
[nltk_data]   Package omw is already up-to-date!
/pytorch/aten/src/ATen/native/LegacyDefinitions.cpp:14: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


Fruta	Fruta	NOUN	_	Case=Abs|Definite=Def	0	root	edible reproductive body of a seed plant especially one having sweet flesh	Fruta:{'edible_fruit'}	https://upload.wikimedia.org/wikipedia/commons/5/5c/Citrus_pulp1.JPG
izozkia	izozki	NOUN	_	Animacy=Inan|Case=Abs|Definite=Def|Number=Sing	3	obj	ice cream or water ice on a small wooden stick	izozkia:{'popsicle', 'ice_lolly', 'lollipop', 'lolly'}	https://upload.wikimedia.org/wikipedia/commons/2/2e/Cucumber%2C_elderflower_and_mint_ice_pop_from_Nicepops_%2818159920902%29.jpg
egiteko	egin	VERB	_	Case=Loc	4	advcl	agree freely	egiteko:{'offer', 'volunteer'}	https://upload.wikimedia.org/wikipedia/commons/6/62/CI_boardwalk_Sandy_sweepers_jeh.jpg
osagaiak	osagai	NOUN	_	Animacy=Inan|Case=Abs|Definite=Def|Number=Plur	1	nsubj	something determined in relation to something that includes it	osagaiak:{'part', 'component', 'component_part', 'constituent', 'portion'}	http://www.image-net.org/synset?wnid=13809207
:	:	PUNCT	_	_	1	punct		_	0
kilo	kilo	NOUN	_	_	20

jan	jan	VERB	_	VerbForm=Part	85	conj	pass through the esophagus as part of eating or drinking	jan:{'get_down', 'swallow'}	https://upload.wikimedia.org/wikipedia/commons/4/4a/Bahama_Swallow.jpg
.	.	PUNCT	_	_	69	punct		_	0
